In [1]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 654.6/654.6 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 54.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 50.4 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.1
    Uninstalling ml-dtypes-0.4.1:
      Successfully uninstalled ml-dtypes-0.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.18.0 requires ml-dtypes<0.5.0,>=0.4.0, but you have ml-dtypes 0.5.1 which is incompatible.
2.20.0-dev20250302


In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2025-03-05 06:19:20--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 104.26.3.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv’

train-data.tsv      100%[===================>] 349.84K  --.-KB/s    in 0.03s   

2025-03-05 06:19:21 (10.2 MB/s) - ‘train-data.tsv’ saved [358233/358233]

--2025-03-05 06:19:21--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 104.26.3.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv’

valid-data.tsv      100%[==============

In [3]:
train_df = pd.read_csv(train_file_path, sep='\t', header=None, names=['label', 'message'])
test_df = pd.read_csv(test_file_path, sep='\t', header=None, names=['label', 'message'])


In [4]:
train_df.head()

,label,message
0,ham,ahhhh...just woken up!had a bad dream about u ...
1,ham,you can never do nothing
2,ham,"now u sound like manky scouse boy steve,like! ..."
3,ham,mum say we wan to go then go... then she can s...
4,ham,never y lei... i v lazy... got wat? dat day ü ...


In [5]:
train_df['label'] = train_df['label'].map({'ham': 0, 'spam': 1})
test_df['label'] = test_df['label'].map({'ham': 0, 'spam': 1})

In [6]:
max_words = 10000
max_len = 150


In [8]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(train_df['message'])



In [9]:
from tensorflow.keras.utils import pad_sequences

In [10]:
X_train = tokenizer.texts_to_sequences(train_df['message'])
X_train = pad_sequences(X_train, maxlen=max_len)

In [11]:
X_test = tokenizer.texts_to_sequences(test_df['message'])
X_test = pad_sequences(X_test, maxlen=max_len)

In [12]:
y_train = train_df['label']
y_test = test_df['label']

In [26]:
from tensorflow.keras import layers, models


model = models.Sequential()
model.add(layers.Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(layers.LSTM(64))
model.add(layers.Dense(1, activation='sigmoid'))

In [27]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [28]:
model.fit(X_train, y_train, epochs=2, batch_size=16, validation_data=(X_test, y_test))

Epoch 1/2
262/262 ━━━━━━━━━━━━━━━━━━━━ 28s 95ms/step - accuracy: 0.9251 - loss: 0.2330 - val_accuracy: 0.9849 - val_loss: 0.0531
Epoch 2/2
262/262 ━━━━━━━━━━━━━━━━━━━━ 23s 88ms/step - accuracy: 0.9949 - loss: 0.0226 - val_accuracy: 0.9878 - val_loss: 0.0453


In [29]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
    seq = tokenizer.texts_to_sequences([pred_text])
    padded_seq = pad_sequences(seq, maxlen=max_len)

    prediction = model.predict(padded_seq)[0][0]

    label = 'ham' if prediction < 0.5 else 'spam'

    return [prediction, label]

In [30]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    print(msg, ans, prediction)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 203ms/step
how are you doing today ham [0.0015636554, 'ham']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
sale today! to stop texts call 98912460324 spam [0.6860566, 'spam']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
i dont want to go. can we try it a different day? available sat ham [0.0048088534, 'ham']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
our new mobile video service is live. just install on your phone to start watching. spam [0.9977353, 'spam']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
you have won £1000 cash! call to claim your prize. spam [0.99536246, 'spam']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
i'll bring it tomorrow. don't forget the milk. ham [0.0024638695, 'ham']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
wow, is your arm alright. that happened to me one time too ham [0.0017259142, 'ham']
You passed the challenge. Great job!
